## Analyze A/B Test Results

You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).  **Please save regularly

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [2]:
# Loading the ab_data.csv.
df = pd.read_csv('ab_data.csv')

# Printing the first 5 rows.
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the below cell to find the number of rows in the dataset.

In [3]:
# Number of rows.
print("Number of rows:", df.shape[0])

Number of rows: 294478


c. The number of unique users in the dataset.

In [4]:
# Number of unique user_id.
print("Number of unique user_id: ", len(df.user_id.unique()))

Number of unique user_id:  290584


d. The proportion of users converted.

In [5]:
# Copying.
df_clean = df.copy()

# Subsetting to remove duplicated user_id.
df_clean = df_clean[np.logical_not(df_clean.user_id.duplicated())]

# Proportion.
sum(df_clean.converted)/len(df_clean)

0.11956955647936569

e. The number of times the `new_page` and `treatment` don't line up.

In [6]:
# treatment_non_line_up
treatment_non_line_up = df.query("group == 'treatment'").landing_page.value_counts()[1]

In [7]:
# new_age_non_line_up
new_age_non_line_up = df.query("landing_page == 'new_page'").group.value_counts()[1]

In [8]:
# Non line-up
print("Non line-up: ", treatment_non_line_up + new_age_non_line_up)

Non line-up:  3893


f. Do any of the rows have missing values?

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [10]:
# Subsetting the non line-up. New Page and Treatment
aux_1 = df.query("landing_page == 'new_page'").query("group == 'treatment'")

# Subsetting the non line-up. Old Page and Control.
aux_2 = df.query("landing_page == 'old_page'").query("group == 'control'")

# Appending by rows.
df2 = aux_1.append(aux_2)

In [11]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [12]:
# Number of unique user_ids
print("Number of Unique user_ids:", df2.user_id.nunique())

Number of Unique user_ids: 290584


b. There is one **user_id** repeated in **df2**.  What is it?

In [13]:
# Number of repeated user_id
print("Number of repeated user_id:", sum(df2.user_id.duplicated()))

Number of repeated user_id: 1


c. What is the row information for the repeat **user_id**? 

In [14]:
# All information about the duplicated user.
df2[df2.user_id.duplicated()].head()

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [15]:
# Repeated User id
df2[df2.user_id.duplicated()].user_id.tolist()[0]

773192

In [16]:
# Landing page of the Repeated user
df2[df2.user_id.duplicated()].landing_page.tolist()[0]

'new_page'

In [17]:
# Group of the repeated User id
df2[df2.user_id.duplicated()].group.tolist()[0]

'treatment'

In [18]:
# Converted of the repeated User id
df2[df2.user_id.duplicated()].converted.tolist()[0]

0

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [19]:
# Removing the duplicated user_id
df2 = df2[np.logical_not(df2.user_id.duplicated())]

# Testing.
print("Number of Duplicated user_id:", sum(df2.user_id.duplicated()))

Number of Duplicated user_id: 0


`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [34]:
# Converting proportion regardless the page.
p_all = sum(df2.converted)/len(df2.converted)

# Printing.
print("Probability:", p_all)

Probability: 0.119597087245


b. Given that an individual was in the `control` group, what is the probability they converted?

In [21]:
# Given it is the control group. What is the converting proportion.
sum(df2.query('group == "control"').converted)/len(df2.query('group == "control"').converted)

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [22]:
# Given it is the treatment group. What is the converting proportion.
sum(df2.query('group == "treatment"').converted)/len(df2.query('group == "treatment"').converted)

0.11880806551510564

d. What is the probability that an individual received the new page?

In [23]:
df2.landing_page.value_counts()[0]/sum(df2.landing_page.value_counts())

0.50006194422266881

e. Use the results in the previous two portions of this question to suggest if you think there is evidence that one page leads to more conversions?  Write your response below.

In [24]:
# Converting timestamp to date time.
df2['timestamp'] = pd.to_datetime(df['timestamp'])

# Calculating the elapsed time from the beginning to the end.
df2.timestamp.max() - df2.timestamp.min()

Timedelta('21 days 23:59:49.081927')

>**Answer:** There is no evidence to support that the new version of the homepage drives a higher conversion rate. Even worse, the control group have performed better results of conversion rates in comparison with the treatment group.
>
>Unfortunately, I do not have much information (a Hypothesis Testing, for instance) only descriptive statistics, but with this preliminary results it is better to keep with the older version.
>
>Bear in mind, this dataset only have data from 21 days, maybe it is too few days for this kind of study, even more, I do not have information about the type of clients have been accessing the website. The older users could have an aversion of this new version of the website and could sabotage it.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

I want to prove the new website has better performance from the older one.

* Null Hypothesis: The old version has at least or equal performance than the newer version.
* Alternative Hypothesis: The newer version is better than the older one.

Converting these statments in Hypoteses Testing:

$$H_0: p_{new} - p_{old} \leq 0 $$
$$H_1: p_{new} - p_{old} > 0$$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

In [ ]:
# Initializing variables
diff = []

# Loop to generate 10,000 samples of 200 samples.
for index in range(10000):
    # Sampling
    sample = df2.sample(200, replace = True)
    
    sample_p_new = sum(df2.query('landing_page == "new_page"').converted)/len(df2.query('landing_page == "new_page"').converted)

    sample_p_old = sum(df2.query('landing_page == "old_page"').converted)/len(df2.query('landing_page == "old_page"').converted)

    sample_diff = sample_p_new - sample_p_old
    diff.append(sample_diff)

a. What is the **convert rate** for $p_{new}$ under the null? 

In [25]:
p_new = sum(df2.query('landing_page == "new_page"').converted)/len(df2.query('landing_page == "new_page"').converted)

print("p_new: ", p_new)

p_new:  0.118808065515


b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [26]:
p_old = sum(df2.query('landing_page == "old_page"').converted)/len(df2.query('landing_page == "old_page"').converted)

print("p_old: ", p_old)

p_old:  0.1203863045


In [33]:
p_new - p_old

-0.0015782389853555567

c. What is $n_{new}$?

In [27]:
n_new = df2.query('landing_page == "new_page"').shape[0]

print("n_new: ", n_new)

n_new:  145310


d. What is $n_{old}$?

In [28]:
n_old = df2.query('landing_page == "old_page"').shape[0]

print("n_old: ", n_old)

n_old:  145274


e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [29]:
new_page_converted = np.random.choice([0, 1], size = n_new, p=[1 - p_new, p_new])

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [30]:
old_page_converted = np.random.choice([0, 1], size = n_old, p=[1 - p_old, p_old])

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [31]:
sum(new_page_converted)/len(new_page_converted) - sum(old_page_converted)/len(old_page_converted)

0.00066551162648804951

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in **p_diffs**.

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

k. In words, explain what you just computed in part **j.**.  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**Put your answer here.**

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [32]:
import statsmodels.api as sm

convert_old = 
convert_new = 
n_old = 
n_new = 

SyntaxError: invalid syntax (<ipython-input-32-349dcc834059>, line 3)

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**Put your answer here.**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Put your answer here.**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a colun for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in the **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

**Put your answer here.**

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**Put your answer here.**

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy varaibles.** Provide the statistical output as well as a written response to answer this question.

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

<a id='conclusions'></a>
## Finishing Up

> Congratulations!  You have reached the end of the A/B Test Results project!  This is the final project in Term 1.  You should be very proud of all you have accomplished!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.


## Directions to Submit

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])